In [67]:
import os
import numpy as np
import pandas as pd
import nibabel as nib
import glob
import matplotlib.pyplot as plt
from PIL import Image
import matplotlib.image as mpimg
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix,precision_score,recall_score

In [ ]:
classes = ['Alzheimer', 'MCI' , 'Normal']

data = {}
labels = {}
for class_ in classes:
    print "working on " + class_ + "..."
    dirs_ = os.listdir('/home/siddhant/SharedFolder/DATA/Data/clean_data/' + class_)
    data_ = []
    for dir_ in dirs_:
        files_path ='/home/siddhant/SharedFolder/DATA/Data/clean_data/' + os.path.join(os.path.join(class_, dir_), '*.nii')
        for file_ in glob.glob(files_path):
            temp_file = nib.load(file_).get_data()
            if temp_file.shape[0] != 64 or temp_file.shape[1] != 64 or temp_file.shape[2] !=48:
                continue
            data_.append(temp_file)
    data[class_] = np.array(data_)

In [ ]:
temp_concat = np.concatenate((data['Alzheimer'][0], data['Alzheimer'][1]), axis=3)
for i in range(2, data['Alzheimer'].shape[0]):
    temp_concat = np.concatenate((temp_concat, data['Alzheimer'][i]), axis=3)
data['Alzheimer'] = temp_concat
print data['Alzheimer'].shape

In [ ]:
# temp_concat = np.concatenate((data['Alzheimer _ 70'][0], data['Alzheimer _ 70'][1]), axis=3)
# for i in range(2, data['Alzheimer _ 70'].shape[0]):
#     temp_concat = np.concatenate((temp_concat, data['Alzheimer _ 70'][i]), axis=3)
# data['Alzheimer _ 70'] = temp_concat
# print data['Alzheimer _ 70'].shape

In [ ]:
temp_concat = np.concatenate((data['MCI'][0], data['MCI'][1]), axis=3)
for i in range(2, data['MCI'].shape[0]):
    temp_concat = np.concatenate((temp_concat, data['MCI'][i]), axis=3)
data['MCI'] = temp_concat
print data['MCI'].shape

In [ ]:
# temp_concat = np.concatenate((data['MCI _ 70'][0], data['MCI _ 70'][1]), axis=3)
# for i in range(2, data['MCI _ 70'].shape[0]):
#     temp_concat = np.concatenate((temp_concat, data['MCI _ 70'][i]), axis=3)
# data['MCI _ 70'] = temp_concat
# print data['MCI _ 70'].shape

In [ ]:
temp_concat = np.concatenate((data['Normal'][0], data['Normal'][1]), axis=3)
for i in range(2, data['Normal'].shape[0]):
    temp_concat = np.concatenate((temp_concat, data['Normal'][i]), axis=3)
data['Normal'] = temp_concat
print data['Normal'].shape

In [ ]:
# temp_concat = np.concatenate((data['Normal _ 70'][0], data['Normal _ 70'][1]), axis=3)
# for i in range(2, data['Normal _ 70'].shape[0]):
#     temp_concat = np.concatenate((temp_concat, data['Normal _ 70'][i]), axis=3)
# data['Normal _ 70'] = temp_concat
# print data['Normal _ 70'].shape

In [15]:
temp_concat = np.concatenate(((normal[0],2), (normal[1],2)), axis=3)

IndexError: axis 3 out of bounds [0, 1)

In [ ]:
np.concatenate((data['Alzheimer'][:,:,:,0],y),axis=3)

In [ ]:
c = map(lambda x,y:(x[],y),data['Alzheimer'],y)

In [ ]:
np.array(c).shape

In [ ]:
np.save('/home/siddhant/SharedFolder/DATA/Normal',data['Normal'])

In [ ]:
np.save('/home/siddhant/SharedFolder/DATA/Alzheimer',data['Alzheimer'])

In [ ]:
np.save('/home/siddhant/SharedFolder/DATA/MCI',data['MCI'])

## Checkpoint


In [2]:
normal=np.load('/home/siddhant/SharedFolder/DATA/Normal.npy')

In [16]:
mci=np.load('/home/siddhant/SharedFolder/DATA/MCI.npy')

In [17]:
alzheimer=np.load('/home/siddhant/SharedFolder/DATA/Alzheimer.npy')

In [24]:
alzheimer.shape

(64, 64, 48, 6440)

In [21]:
temp_concat = np.concatenate((normal[0], normal[1]), axis=3)
for i in range(2, normal.shape[0]):
    temp_concat = np.concatenate((temp_concat, normal[i]), axis=3)
normal = temp_concat
print normal.shape

(64, 64, 48, 6300)


In [22]:
temp_concat = np.concatenate((mci[0], mci[1]), axis=3)
for i in range(2, mci.shape[0]):
    temp_concat = np.concatenate((temp_concat, mci[i]), axis=3)
mci = temp_concat
print mci.shape

(64, 64, 48, 5320)


In [35]:
data={}
data_=[]
for i in range(normal.shape[3]):
    data_.append((normal[:,:,:,i],2))
data['Normal'] = data_
del(normal)

In [36]:
data_=[]
for i in range(alzheimer.shape[3]):
    data_.append((alzheimer[:,:,:,i],0))
data['Alzheimer'] = data_
del(alzheimer)

In [37]:
data_=[]
for i in range(mci.shape[3]):
    data_.append((mci[:,:,:,i],1))
data['MCI'] = data_
del(data_)
del(mci)

In [69]:
combined_data = data['MCI'] + data['Alzheimer'] + data['Normal']

In [81]:
np.array(X).shape

(18060, 64, 64, 48)

### if you can then find a better way to make labels and shuffle data

In [71]:
random.shuffle(combined_data)

In [72]:
df=pd.DataFrame({'combined': combined_data})

In [77]:
df[['X', 'y']] = df['combined'].apply(pd.Series)

In [78]:
df.head()


,combined,X,y
0,([[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,"[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",1
1,([[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,"[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",1
2,([[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,"[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",1
3,([[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,"[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",1
4,([[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,"[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",0


In [79]:
X=df['X'].tolist()

In [80]:
y=df['y'].tolist()

In [88]:
del(df)

## Neural Network

###  Prepare Data

In [82]:
x_Train, x_val, y_Train, y_val = train_test_split(X,y, test_size = 0.1)

In [84]:
x_train, x_test, y_train, y_test = train_test_split(x_Train,y_Train, test_size = 0.1)

In [186]:
np.array(y_val).shape

(1806,)

In [191]:
short_x_train=x_train[:1000]
short_y_train=y_train[:1000]

In [192]:
short_x_val=x_val[:100]
short_y_val=y_val[:100]

In [193]:
short_x_test=x_test[:1000]
short_y_test=y_test[:1000]

AttributeError: 'list' object has no attribute 'shape'

In [182]:
def batch_generator(X, y, batch_size,samples_per_epoch):
    number_of_batches = (samples_per_epoch)/batch_size
    counter=0
    shuffle_index = np.arange(np.shape(y)[0])
    np.random.shuffle(shuffle_index)
    X =  X[shuffle_index,:]
    y =  y[shuffle_index]
    while 1:
        index_batch = shuffle_index[batch_size*counter:batch_size*(counter+1)]
        X_batch = X[index_batch,:]
        y_batch = y[index_batch]
        counter += 1
        yield(np.array(X_batch),y_batch)
        if (counter < number_of_batches):
            np.random.shuffle(shuffle_index)
            counter=0

In [174]:
# import the necessary packages
from keras.models import Sequential
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
 
# initialize the model
model = Sequential()

#first layer
model.add(Convolution2D(20, 5, 5, border_mode="same",
    input_shape=(64, 64, 48)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

#second layer
model.add(Convolution2D(50, 5, 5, border_mode="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

#set of FC
model.add(Flatten())
model.add(Dense(500))
model.add(Activation("relu"))

# softmax classifier
model.add(Dense(3))
model.add(Activation("softmax"))

#get model summary
model.summary()

# Compile model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

/home/siddhant/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:14: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(20, (5, 5), padding="same", input_shape=(64, 64, 4...)`
/home/siddhant/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:19: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(50, (5, 5), padding="same")`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           (None, 64, 64, 20)        24020     
_________________________________________________________________
activation_28 (Activation)   (None, 64, 64, 20)        0         
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 32, 32, 20)        0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 32, 32, 50)        25050     
_________________________________________________________________
activation_29 (Activation)   (None, 32, 32, 50)        0         
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 16, 16, 50)        0         
_________________________________________________________________
flatten_8 (Flatten)          (None, 12800)             0         
__________

In [ ]:
model.fit_generator(generator=batch_generator(np.asarray(short_x_train), np.asarray(short_y_train), 256,np.array(short_x_train).shape[0]),
                    nb_epoch=5, samples_per_epoch=np.array(short_x_train).shape[0],
                    validation_data=batch_generator(np.asarray(short_x_val), np.asarray(short_y_val), 256,np.array(short_x_val).shape[0]),
                    nb_val_samples=np.array(short_x_val).shape[0])

/home/siddhant/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:4: UserWarning: The semantics of the Keras 2 argument  `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Update your method calls accordingly.
/home/siddhant/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:4: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(generator=<generator..., validation_data=<generator..., steps_per_epoch=1000, epochs=5, validation_steps=100)`


Epoch 1/5
   1/1000 [..............................] - ETA: 23552s - loss: 11.7108 - acc: 0.2734